In [0]:
INBOUND_PATH = "/mnt/yahoo-finances-bg/inbound/"

FILE_TEST_PATH = "/FileStore/tables/finances_AAPL_21*.csv"

RAW_PATH = "/mnt/yahoo-finances-bg/raw/"

PROCESSING_PATH = "/mnt/yahoo-finances-bg/processing/"

In [0]:
get_sch_data = spark.read.format('csv').option('header', 'true').load(FILE_TEST_PATH)
data_static_sch = get_sch_data.schema

get_sch_data.createOrReplaceTempView("vw_data")

In [0]:
df_bronze = spark.readStream \
 .schema(data_static_sch) \
 .format("parquet") \
 .option("header", "true") \
 .option("maxFilesPerTrigger", "1") \
 .load(RAW_PATH)

df_bronze.isStreaming

Out[3]: True

In [0]:
df_bronze \
 .writeStream \
 .format("delta") \
 .outputMode("append") \
 .partitionBy("year", "month", "ticker") \
 .option("checkpointLocation", PROCESSING_PATH + "_checkpoint/") \
 .start(PROCESSING_PATH)

Out[4]: <pyspark.sql.streaming.StreamingQuery at 0x7f2038459650>

In [0]:
%sql 

SELECT * FROM delta.`/mnt/yahoo-finances-bg/processing`

open,close,high,volume,low,timestamp,day,year,month,ticker
16.125,6417722,16.3799991607666,16.229999542236328,16.3700008392334,1642689000,21,2022,1,VALE
16.21500015258789,3358880,16.309999465942383,16.250699996948242,16.229999542236328,1642692600,21,2022,1,VALE
16.235000610351562,3779589,16.3700008392334,16.3700008392334,16.2549991607666,1642696200,21,2022,1,VALE
16.300100326538086,2881656,16.399999618530273,16.334999084472656,16.364999771118164,1642699800,21,2022,1,VALE
16.280000686645508,4782015,16.3799991607666,16.283599853515625,16.334999084472656,1642703400,21,2022,1,VALE
16.020000457763672,6965979,16.28499984741211,16.02669906616211,16.28499984741211,1642707000,21,2022,1,VALE
15.9399995803833,6554616,16.04990005493164,15.989999771118164,16.030000686645508,1642710600,21,2022,1,VALE
166.97999572753906,169.22999572753906,169.33999633789062,19181838,166.75999450683594,1642689000,21,2022,1,AAPL
169.23989868164062,169.35000610351562,169.63999938964844,9252283,168.94000244140625,1642692600,21,2022,1,AAPL
169.36000061035156,169.10000610351562,169.47999572753906,8884453,168.77999877929688,1642696200,21,2022,1,AAPL
